In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import statistics as st

%matplotlib inline  

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a'
api_key = 'c81352430e2fe3c941faf0814227562b'

data

In [4]:
''' Колонки '''
ticker = 'AAPL'
# ticker = 'AAPL'

# tickers_list = ['ADBE', 'COP', 'AMAT']
# tickers_list = ['F', 'WU', 'KMI', 'CNP', 'HPQ', 'GLW', 'KHC', 'LUV', 'CMCSA', 'NRG', 'BK', 'CSCO', 'USB', 'TJX', 'CAH', 'HIG', 'SCHW', 'HAS', 'WELL', 'DXCM', 'ROST', 'GRMN', 'NKE', 'YUM', 'GOOGL', 'JPM', 'GOOG', 'GPN', 'MMM', 'AXP', 'TGT', 'TXN', 'CRM', 'NDAQ', 'CAT', 'ILMN', 'V', 'GD', 'NFLX', 'MSFT', 'HD', 'MA', 'SPGI', 'IDXX', 'ADBE', 'AVGO',
# 'UAA', 'HBAN', 'HPE', 'IVZ', 'KEY', 'RF', 'BEN', 'WMB', 'DAL', 'GM', 'WRK', 'WFC', 'EBAY', 'VTR', 'AIG', 'MU', 'MET', 'NTAP', 'SYY', 'BXP', 'MNST', 'XOM', 'PYPL', 'DIS', 'TROW', 'HLT', 'QCOM', 'CVX', 'AAPL', 'HON', 'UPS']
tickers_list = ['LRCX', 'BKNG','KEY', 'BAC', 'LVS', 'UAL', 'HOG', 'CFG', 'ALK', 'MAS', 'BBY', 'EMR', 'PSX', 'MS', 'LYB', 'RTX', 'RL', 'COP', 'AMAT', 'QRVO', 'SPG', 'DRI', 'BA', 'META', 'NVDA', 'FDX', 'ADSK', 'ABMD', 'GS',
'GPS', 'XRX', 'BKR', 'PARA', 'FCX', 'ALK', 'WDC', 'C', 'MOS', 'OKE', 'DHI', 'BBY', 'MPC', 'AMD', 'PRU', 'EXPE', 'EOG', 'VLO', 'MAR', 'HCA', 'ALB']

required_year = '2022'


''' время '''
stime='1990-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [5]:
def one_stock_analyse(ticker, api_key, required_year):

    def get_data_from_ticker(tick, interval='1mo'):
        ticker = yf.Ticker(tick)

        df = ticker.history(start='1990-01-01', end=None, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    
    df = get_data_from_ticker(ticker)



    def change_index(df):
        df_index = df.index.to_list()
        for i in range(len(df_index)):
            t = df_index[i].date()
            df_index[i] = t.strftime('%Y-%m-%d')
        df['Date_index'] = df_index
        df = df.set_index('Date_index')
        return df

    df = change_index(df)


    def get_key_metricks(ticker, api_key, required_year):
        key_metrics = fa.key_metrics(ticker, api_key, period='annual')
        df_key_metr = key_metrics.T
        df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
        return df_key_metr



    df_key_metr = get_key_metricks(ticker, api_key, required_year)


    if df_key_metr.index[0] == required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
        last_year = True
    else:
        last_year = False



    def get_index(df):
        index_years = df.index.to_list()
        return index_years

    index_years = get_index(df_key_metr)



    def get_price_from_df(df, list_dates):
        new_list = []
        for i in range(len(list_dates)):
            x = int(list_dates[i])
            x += 1
            x = str(x)
            try:
                r = df.loc[f'{x}-01-01']
                r = float(r)
                new_list.append(r)
            except:
                new_list.append(0)

        return new_list


    stock_year_prices = get_price_from_df(df, index_years)


    df_key_metr[ticker] = list(stock_year_prices)
    df_key_metr.drop('period', axis=1, inplace=True)


    def change_percent_all_columns(df):
        x = df.columns.to_list()
        for i in range(len(x)):
            v = df[x[i]].to_list()
            l = []
            for y in range(len(v)):
                try:
                    if y != len(v):
                        vv = (v[y] / (v[y+1] / 100)) - 100
                        l.append(vv)
                    else:
                        vv = 0
                        l.append(vv)
                except:
                    l.append(0)
            df[f'change % {x[i]}'] = l
        return df

    df_with_changes = change_percent_all_columns(df_key_metr)
    df_with_changes = df_with_changes.reindex(index=df_with_changes.index[::-1])


    df_some_columns = df_with_changes[[f'change % {ticker}', 'change % debtToAssets']]


    mean_result = st.mean(df_some_columns[f'change % {ticker}'].to_list()[1:])    
    
    l = []

    for i in range(len(df_some_columns['change % debtToAssets'])):
        if i <= 1:
            pass
        elif i == len(df_some_columns['change % debtToAssets']) - 1:
            pass
        elif df_with_changes[f'change % {ticker}'][i] < df_with_changes[f'change % {ticker}'][i-1] and df_with_changes['change % debtToAssets'][i] > df_with_changes['change % debtToAssets'][i-1] and df_with_changes['change % debtToAssets'][i] > df_with_changes[f'change % {ticker}'][i]:
            l.append(df_with_changes[f'change % {ticker}'][i+1])
        
        else:
            pass  
    
    return [ticker, mean_result, st.mean(l)]

Mean results

In [6]:
def list_stock_analyse(tickers_list, api_key, required_year):
    l = []
    for i in range(len(tickers_list)):
        try:
            l.append(one_stock_analyse(tickers_list[i], api_key, required_year))
        except:
            pass
    return l


df_means_results = pd.DataFrame(list_stock_analyse(tickers_list, api_key, required_year), columns = ['Ticker', 'Mean', 'Str_mean'])
df_means_results.head(3)

,Ticker,Mean,Str_mean
0,LRCX,23.235716,30.619577
1,BKNG,45.252898,5.496889
2,KEY,8.064269,18.075659


In [7]:


df_means_results['Mean'].mean()
df_means_results['Str_mean'].mean()

11.269657765674985

In [8]:
def get_data_from_ticker(tick, interval='1mo'):
        ticker = yf.Ticker(tick)

        df = ticker.history(start='1990-01-01', end=None, interval=interval)
        x = pd.DataFrame(df)
        x.rename(columns={"Close": tick}, inplace=True)
        z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
        return z

    
df = get_data_from_ticker(ticker)



def change_index(df):
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        t = df_index[i].date()
        df_index[i] = t.strftime('%Y-%m-%d')
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

df = change_index(df)


def get_key_metricks(ticker, api_key, required_year):
    key_metrics = fa.key_metrics(ticker, api_key, period='annual')
    df_key_metr = key_metrics.T
    df_key_metr = df_key_metr.loc[required_year:'2000'].copy(deep=True)
    return df_key_metr

    
    
df_key_metr = get_key_metricks(ticker, api_key, required_year)
    
    
if df_key_metr.index[0] == required_year and f'{int(required_year)+1}-01-01' in df.index.to_list():
    last_year = True
else:
    last_year = False



def get_index(df):
    index_years = df.index.to_list()
    return index_years

index_years = get_index(df_key_metr)



def get_price_from_df(df, list_dates):
    new_list = []
    for i in range(len(list_dates)):
        x = int(list_dates[i])
        x += 1
        x = str(x)
        try:
            r = df.loc[f'{x}-01-01']
            r = float(r)
            new_list.append(r)
        except:
            new_list.append(0)
                
    return new_list

    
stock_year_prices = get_price_from_df(df, index_years)


df_key_metr[ticker] = list(stock_year_prices)
df_key_metr.drop('period', axis=1, inplace=True)

    
def change_percent_all_columns(df):
    x = df.columns.to_list()
    for i in range(len(x)):
        v = df[x[i]].to_list()
        l = []
        for y in range(len(v)):
            try:
                if y != len(v):
                    vv = (v[y] / (v[y+1] / 100)) - 100
                    l.append(vv)
                else:
                    vv = 0
                    l.append(vv)
            except:
                l.append(0)
        df[f'change % {x[i]}'] = l
    return df

df_with_changes = change_percent_all_columns(df_key_metr)
df_with_changes = df_with_changes.reindex(index=df_with_changes.index[::-1])

    
df_some_columns = df_with_changes[[f'change % {ticker}', 'change % debtToAssets']]
    
    
mean_result = st.mean(df_some_columns[f'change % {ticker}'].to_list()[1:])    
    


ValueError: This endpoint is only for premium members. Please visit the subscription page to upgrade the plan (Starter or higher) at https://financialmodelingprep.com/developer/docs/pricing

In [ ]:
# df_some_columns['Date'] = 

In [ ]:
# df1 = df_some_columns.copy()
# df_some_columns['Date'] = df1.index.to_list()


df_some_columns.index.name='Date'
df_some_columns = df_some_columns.reset_index()
df_some_columns


result

In [ ]:
'''получаем среднее значение акции за указанный период'''
mean_result

In [ ]:
'''получаем значения акции и средняя от стратегии'''

l = []
l1 = []
for i in range(len(df_some_columns['change % debtToAssets'])):
    if i <= 1:
        pass
    elif i == len(df_some_columns['change % debtToAssets']) - 1:
        pass
    elif df_with_changes[f'change % {ticker}'][i] < df_with_changes[f'change % {ticker}'][i-1] and df_with_changes['change % debtToAssets'][i] > df_with_changes['change % debtToAssets'][i-1] and df_with_changes['change % debtToAssets'][i] > df_with_changes[f'change % {ticker}'][i]:
        x = df_with_changes['change % debtToAssets'][i] - df_with_changes[f'change % {ticker}'][i]
        l.append(df_with_changes[f'change % {ticker}'][i+1])
        l1.append(x)          
    else:
        pass     

st.mean(l)
                 

                 
x = {st.mean(l):[l,l1]}
x

In [ ]:
''' получаем сколько значения было верно, сколько нет'''
rights_l = []

for i in range(len(l)):
    if l[i] > 0:
        rights_l.append(True)
    else:
        rights_l.append(False)
    
rights_l


In [ ]:
# df_some_columns.head(3)

In [ ]:
# def get_results():